In [1]:
import numpy as np
import cv2 as cv
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import warnings
warnings.filterwarnings('ignore')

## Mission
 - train data로 model 만들기
 - 이미지 파일 읽어서 그걸 모델의 입력값으로 줘서 예측값 얻어내기

In [2]:
df_train_raw = pd.read_csv('train_vision.csv')
df_train_raw.head(5)

,filename,label
0,face_3267.png,1
1,face_587.png,4
2,face_6585.png,4
3,face_3094.png,4
4,face_1170.png,4


In [3]:
train_imgfiles = df_train_raw['filename']
train_img_data = []

for train_img in train_imgfiles:
    img = cv.imread('img/faces_images/'+train_img)
    train_img_data.append(img)
    
train_img_data = np.array(train_img_data)
train_img_data.shape

(5850, 128, 128, 3)

In [4]:
train_label = df_train_raw[['label']]-1
train_label.shape

(5850, 1)

### test img 만들기

In [5]:
df_test_raw = pd.read_csv('test_vision.csv')
df_test_raw.head(5)

,filename
0,face_2624.png
1,face_3385.png
2,face_5015.png
3,face_6715.png
4,face_6171.png


In [6]:
test_imgfiles = df_test_raw['filename']
test_img_data = []

for test_img in test_imgfiles:
    img = cv.imread('img/faces_images/'+test_img)
    test_img_data.append(img)
    
test_img_data = np.array(test_img_data)
test_img_data.shape

(2000, 128, 128, 3)

## 모델 정의

In [8]:
#parameter
learning_rate = 0.001
training_epochs = 100
batch_size = 100
nb_classes = 6 # 0~5

tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, 128, 128, 3])
Y = tf.placeholder(tf.int32,[None, 1])
Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print('one-hot : ', Y_one_hot)

#dropout rate 0.7 on training, but should be1 for testing
keep_prob = tf.placeholder(tf.float32)

W0 = tf.Variable(tf.random_normal([5,5,3,12], stddev = 0.01))
L0 = tf.nn.conv2d(X, W0, strides=[1,1,1,1], padding='SAME')
print('L0_0 : ', L0)
L0 = tf.nn.relu(L0)
L0 = tf.nn.max_pool(L0, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L0 = tf.nn.dropout(L0, keep_prob=keep_prob)
print('L0_1 : ', L0)

W1 = tf.Variable(tf.random_normal([5,5,12,24], stddev = 0.01))
L1 = tf.nn.conv2d(L0, W1, strides=[1,1,1,1], padding='SAME')
print('L1_0 : ', L1)
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
print('L1_1 : ', L1)


W2 = tf.Variable(tf.random_normal([5,5,24,72], stddev = 0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
print('L2_0 : ', L2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)
print('L2_1 : ', L2)


L2_flat = tf.layers.flatten(L2)
print('L2_flat : ', L2_flat)


W3 = tf.Variable(tf.random_normal([16*16*72,nb_classes], stddev=0.1))
b = tf.Variable(tf.random_normal([nb_classes]))

with tf.device('/GPU:0'):
    logits = tf.matmul(L2_flat, W3) + b
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## weight 저장하기 
SAVER_DIR = 'model'
saver = tf.train.Saver()
checkpoint_path = os.path.join(SAVER_DIR, 'model')
ckpt = tf.train.get_checkpoint_state(SAVER_DIR)

gpu_options=tf.GPUOptions(visible_device_list="0", per_process_gpu_memory_fraction=0.4)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

one-hot :  Tensor("Reshape:0", shape=(?, 6), dtype=float32)
L0_0 :  Tensor("Conv2D:0", shape=(?, 128, 128, 12), dtype=float32)
L0_1 :  Tensor("dropout/mul:0", shape=(?, 64, 64, 12), dtype=float32)
L1_0 :  Tensor("Conv2D_1:0", shape=(?, 64, 64, 24), dtype=float32)
L1_1 :  Tensor("dropout_1/mul:0", shape=(?, 32, 32, 24), dtype=float32)
L2_0 :  Tensor("Conv2D_2:0", shape=(?, 32, 32, 72), dtype=float32)
L2_1 :  Tensor("dropout_2/mul:0", shape=(?, 16, 16, 72), dtype=float32)
L2_flat :  Tensor("flatten/Reshape:0", shape=(?, 18432), dtype=float32)


## 학습 및 모델저장

In [ ]:
#initialize & saver
sess.run(tf.global_variables_initializer())


avg_cost_array = []     # 그래프 그리기용 list
for epoch in range(training_epochs):
    avg_cost=0
    
    total_batch = int(len(train_img_data) / batch_size)
    for i in range(total_batch):
        batch_xs = train_img_data[batch_size*i : (i+1)*batch_size]
        batch_ys = train_label[batch_size*i : (i+1)*batch_size]
#         print('batch_xs : ', batch_xs.shape, '\nbatch_ys : ', batch_ys.shape)
        saver.save(sess, checkpoint_path, global_step=batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    avg_cost_array.append(avg_cost)
    
    if (epoch+1) == 1:
        print('Epoch : ', '%04d' %(epoch + 1), 'cost = ','{:.9f}'.format(avg_cost))
    if (epoch+1)%10 == 0:
        print('Epoch : ', '%04d' %(epoch + 1), 'cost = ','{:.9f}'.format(avg_cost))
        
print('Learning Finished')
plt.plot(range(training_epochs), avg_cost_array, 'b-')
plt.legend(['Cost'])
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.show();

## Testing

In [27]:
## testing & 출력 label DataFrame 만들기
output_label_list = []
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    for i in range(len(test_img_data)):
        output_label = sess.run(tf.argmax(logits,1) + 1 , feed_dict={X:test_img_data[i:i+1], keep_prob:1.0})
        print('Prediction : ', test_imgfiles[i],' / ', output_label)
        output_label_list.append(output_label)

INFO:tensorflow:Restoring parameters from model\model-100
Prediction :  face_2624.png  /  [1]
Prediction :  face_3385.png  /  [4]
Prediction :  face_5015.png  /  [1]
Prediction :  face_6715.png  /  [1]
Prediction :  face_6171.png  /  [4]
Prediction :  face_3269.png  /  [4]
Prediction :  face_4942.png  /  [4]
Prediction :  face_1095.png  /  [5]
Prediction :  face_7721.png  /  [4]
Prediction :  face_2455.png  /  [1]
Prediction :  face_6595.png  /  [4]
Prediction :  face_2024.png  /  [5]
Prediction :  face_1232.png  /  [4]
Prediction :  face_3796.png  /  [2]
Prediction :  face_4859.png  /  [4]
Prediction :  face_67.png  /  [4]
Prediction :  face_5667.png  /  [4]
Prediction :  face_2672.png  /  [1]
Prediction :  face_6755.png  /  [4]
Prediction :  face_5293.png  /  [2]
Prediction :  face_3996.png  /  [1]
Prediction :  face_7237.png  /  [4]
Prediction :  face_925.png  /  [4]
Prediction :  face_566.png  /  [4]
Prediction :  face_2910.png  /  [4]
Prediction :  face_266.png  /  [5]
Prediction 

Prediction :  face_2605.png  /  [4]
Prediction :  face_6505.png  /  [1]
Prediction :  face_1721.png  /  [4]
Prediction :  face_3686.png  /  [1]
Prediction :  face_4196.png  /  [3]
Prediction :  face_4191.png  /  [4]
Prediction :  face_3848.png  /  [2]
Prediction :  face_6971.png  /  [4]
Prediction :  face_7091.png  /  [1]
Prediction :  face_6192.png  /  [1]
Prediction :  face_138.png  /  [2]
Prediction :  face_6451.png  /  [4]
Prediction :  face_6978.png  /  [4]
Prediction :  face_4005.png  /  [1]
Prediction :  face_7989.png  /  [1]
Prediction :  face_4560.png  /  [1]
Prediction :  face_1919.png  /  [1]
Prediction :  face_7523.png  /  [5]
Prediction :  face_1731.png  /  [6]
Prediction :  face_6471.png  /  [1]
Prediction :  face_5955.png  /  [6]
Prediction :  face_1786.png  /  [6]
Prediction :  face_7694.png  /  [1]
Prediction :  face_7424.png  /  [4]
Prediction :  face_6511.png  /  [1]
Prediction :  face_514.png  /  [6]
Prediction :  face_1126.png  /  [1]
Prediction :  face_1990.png  /

Prediction :  face_6373.png  /  [6]
Prediction :  face_2033.png  /  [1]
Prediction :  face_3914.png  /  [4]
Prediction :  face_659.png  /  [4]
Prediction :  face_6473.png  /  [4]
Prediction :  face_5708.png  /  [4]
Prediction :  face_1091.png  /  [4]
Prediction :  face_1858.png  /  [4]
Prediction :  face_528.png  /  [4]
Prediction :  face_1442.png  /  [4]
Prediction :  face_4214.png  /  [4]
Prediction :  face_3940.png  /  [2]
Prediction :  face_149.png  /  [4]
Prediction :  face_78.png  /  [1]
Prediction :  face_4346.png  /  [2]
Prediction :  face_321.png  /  [4]
Prediction :  face_5831.png  /  [5]
Prediction :  face_109.png  /  [6]
Prediction :  face_2204.png  /  [4]
Prediction :  face_1229.png  /  [4]
Prediction :  face_246.png  /  [4]
Prediction :  face_2630.png  /  [4]
Prediction :  face_7386.png  /  [6]
Prediction :  face_6073.png  /  [4]
Prediction :  face_6004.png  /  [4]
Prediction :  face_3975.png  /  [4]
Prediction :  face_2926.png  /  [6]
Prediction :  face_577.png  /  [5]
P

Prediction :  face_3177.png  /  [4]
Prediction :  face_1012.png  /  [6]
Prediction :  face_2536.png  /  [1]
Prediction :  face_6010.png  /  [1]
Prediction :  face_4290.png  /  [4]
Prediction :  face_315.png  /  [2]
Prediction :  face_469.png  /  [6]
Prediction :  face_6102.png  /  [4]
Prediction :  face_7695.png  /  [4]
Prediction :  face_362.png  /  [2]
Prediction :  face_2470.png  /  [3]
Prediction :  face_1759.png  /  [5]
Prediction :  face_2601.png  /  [4]
Prediction :  face_5027.png  /  [5]
Prediction :  face_7949.png  /  [1]
Prediction :  face_3394.png  /  [1]
Prediction :  face_2291.png  /  [4]
Prediction :  face_5605.png  /  [1]
Prediction :  face_458.png  /  [2]
Prediction :  face_5986.png  /  [1]
Prediction :  face_1890.png  /  [1]
Prediction :  face_3955.png  /  [6]
Prediction :  face_2268.png  /  [2]
Prediction :  face_4034.png  /  [4]
Prediction :  face_7232.png  /  [1]
Prediction :  face_5341.png  /  [4]
Prediction :  face_7338.png  /  [4]
Prediction :  face_756.png  /  [

Prediction :  face_2340.png  /  [4]
Prediction :  face_2684.png  /  [4]
Prediction :  face_3596.png  /  [4]
Prediction :  face_4140.png  /  [4]
Prediction :  face_763.png  /  [6]
Prediction :  face_592.png  /  [1]
Prediction :  face_5176.png  /  [2]
Prediction :  face_4349.png  /  [4]
Prediction :  face_6090.png  /  [4]
Prediction :  face_626.png  /  [4]
Prediction :  face_4172.png  /  [4]
Prediction :  face_4782.png  /  [1]
Prediction :  face_5421.png  /  [4]
Prediction :  face_5792.png  /  [2]
Prediction :  face_4670.png  /  [4]
Prediction :  face_2967.png  /  [5]
Prediction :  face_4072.png  /  [4]
Prediction :  face_1224.png  /  [6]
Prediction :  face_1788.png  /  [1]
Prediction :  face_7079.png  /  [1]
Prediction :  face_174.png  /  [5]
Prediction :  face_6845.png  /  [4]
Prediction :  face_2950.png  /  [4]
Prediction :  face_5178.png  /  [1]
Prediction :  face_6900.png  /  [1]
Prediction :  face_7469.png  /  [1]
Prediction :  face_3864.png  /  [1]
Prediction :  face_864.png  /  [

Prediction :  face_2986.png  /  [5]
Prediction :  face_7719.png  /  [1]
Prediction :  face_2365.png  /  [1]
Prediction :  face_4519.png  /  [3]
Prediction :  face_6096.png  /  [4]
Prediction :  face_157.png  /  [4]
Prediction :  face_6915.png  /  [4]
Prediction :  face_6630.png  /  [1]
Prediction :  face_125.png  /  [1]
Prediction :  face_7209.png  /  [1]
Prediction :  face_581.png  /  [1]
Prediction :  face_3200.png  /  [1]
Prediction :  face_5594.png  /  [4]
Prediction :  face_5065.png  /  [5]
Prediction :  face_623.png  /  [1]
Prediction :  face_1175.png  /  [4]
Prediction :  face_7272.png  /  [4]
Prediction :  face_5699.png  /  [2]
Prediction :  face_7585.png  /  [3]
Prediction :  face_488.png  /  [1]
Prediction :  face_3406.png  /  [4]
Prediction :  face_3510.png  /  [4]
Prediction :  face_7972.png  /  [4]
Prediction :  face_1200.png  /  [2]
Prediction :  face_1750.png  /  [4]
Prediction :  face_7205.png  /  [4]
Prediction :  face_4352.png  /  [1]
Prediction :  face_1472.png  /  [

Prediction :  face_7700.png  /  [4]
Prediction :  face_3391.png  /  [1]
Prediction :  face_710.png  /  [4]
Prediction :  face_891.png  /  [4]
Prediction :  face_2320.png  /  [1]
Prediction :  face_119.png  /  [1]
Prediction :  face_2224.png  /  [1]
Prediction :  face_536.png  /  [5]
Prediction :  face_1608.png  /  [5]
Prediction :  face_5415.png  /  [1]
Prediction :  face_5140.png  /  [4]
Prediction :  face_3124.png  /  [4]
Prediction :  face_3581.png  /  [4]
Prediction :  face_1881.png  /  [2]
Prediction :  face_839.png  /  [4]
Prediction :  face_4402.png  /  [4]
Prediction :  face_2760.png  /  [5]
Prediction :  face_2476.png  /  [1]
Prediction :  face_7308.png  /  [5]
Prediction :  face_359.png  /  [1]
Prediction :  face_2093.png  /  [6]
Prediction :  face_6492.png  /  [4]
Prediction :  face_6827.png  /  [2]
Prediction :  face_6925.png  /  [1]
Prediction :  face_1719.png  /  [2]
Prediction :  face_2742.png  /  [1]
Prediction :  face_7240.png  /  [2]
Prediction :  face_7729.png  /  [1

Prediction :  face_1626.png  /  [1]
Prediction :  face_3649.png  /  [5]
Prediction :  face_2594.png  /  [1]
Prediction :  face_3542.png  /  [5]
Prediction :  face_5249.png  /  [4]
Prediction :  face_2210.png  /  [6]
Prediction :  face_7435.png  /  [5]
Prediction :  face_5492.png  /  [1]
Prediction :  face_3023.png  /  [6]
Prediction :  face_4794.png  /  [4]
Prediction :  face_2876.png  /  [4]
Prediction :  face_1934.png  /  [4]
Prediction :  face_6195.png  /  [1]
Prediction :  face_3812.png  /  [6]
Prediction :  face_5546.png  /  [5]
Prediction :  face_1311.png  /  [6]
Prediction :  face_5871.png  /  [2]
Prediction :  face_5939.png  /  [1]
Prediction :  face_1301.png  /  [2]
Prediction :  face_5862.png  /  [5]
Prediction :  face_2354.png  /  [1]
Prediction :  face_2111.png  /  [1]
Prediction :  face_5741.png  /  [5]
Prediction :  face_1492.png  /  [4]
Prediction :  face_5734.png  /  [4]
Prediction :  face_1110.png  /  [4]
Prediction :  face_3835.png  /  [3]
Prediction :  face_6580.png 

Prediction :  face_1943.png  /  [6]
Prediction :  face_6024.png  /  [4]
Prediction :  face_5035.png  /  [4]
Prediction :  face_5007.png  /  [6]
Prediction :  face_4182.png  /  [1]
Prediction :  face_3701.png  /  [1]
Prediction :  face_7189.png  /  [5]
Prediction :  face_1536.png  /  [4]
Prediction :  face_6127.png  /  [4]
Prediction :  face_3943.png  /  [4]
Prediction :  face_5563.png  /  [4]
Prediction :  face_1424.png  /  [4]
Prediction :  face_2157.png  /  [1]
Prediction :  face_1031.png  /  [2]
Prediction :  face_4450.png  /  [4]
Prediction :  face_2775.png  /  [1]
Prediction :  face_2230.png  /  [1]
Prediction :  face_335.png  /  [4]
Prediction :  face_5863.png  /  [4]
Prediction :  face_7389.png  /  [4]
Prediction :  face_5703.png  /  [4]
Prediction :  face_5081.png  /  [4]
Prediction :  face_1852.png  /  [5]
Prediction :  face_1870.png  /  [4]
Prediction :  face_1237.png  /  [4]
Prediction :  face_6757.png  /  [5]
Prediction :  face_1264.png  /  [5]
Prediction :  face_2179.png  

In [40]:
#csv 파일로 저장하기 
output_label_list = pd.DataFrame(output_label_list, columns=['prediction'])
# output_label_list
output_label_list.to_csv('test.csv', index=False)